# Data Exploration

This notebook is meant to be a template for exploration of data collected from trials.

## Before running the notebook
The trial datastore process must be started prior to running the notebook. Make sure you open a terminal, activate the `.venv` environment, then run:
```
python -m launch_local_services
```

Once you are done with working in this notebook, close the terminal window.

## Cogment Setup

In [ ]:
import os
import sys

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "../"))
sys.path.append(ROOT_DIR)

from cogment_verse.utils.generate import generate
from cogment_verse.app import SPEC_FILEPATH

WORK_DIR = os.path.join(ROOT_DIR, ".cogment_verse")

generate(WORK_DIR, SPEC_FILEPATH)

In [ ]:
import asyncio
import functools
import re
from dataclasses import dataclass, field
from datetime import datetime
from typing import List

import cogment
import numpy as np
import pandas as pd
from cogment.control import TrialState
from tabulate import tabulate
from utils import get_trial_ids, get_trial_data

from cogment_verse.specs import (
    AgentConfig,
    cog_settings,
    EnvironmentConfig,
    EnvironmentSpecs,
    HUMAN_ACTOR_IMPL,
    PLAYER_ACTOR_CLASS,
    TEACHER_ACTOR_CLASS,
    WEB_ACTOR_NAME,
)
from cogment_verse.specs.ndarray_serialization import deserialize_ndarray, serialize_ndarray


TRIAL_DATASTORE_ENDPOINT = "grpc://localhost:9001"

## Extract trial ids from datastore

In [ ]:
trial_ids = await get_trial_ids(TRIAL_DATASTORE_ENDPOINT)
print(f"Trial ids count: {len(trial_ids)}")
print(f"first 10 trial ids: [{', '.join(trial_ids[:10])}]")

## Extract Trial Data

Example extracting trial info and `samples` for all trials in the datastore.

If only interested in a single or specific set of trial ids, simply pass elements in the list argument `trial_ids` of the function `get_trial_data`.

In [ ]:
trial_data = await get_trial_data(TRIAL_DATASTORE_ENDPOINT, trial_ids=[])

To retrieve the info and samples from a specific trial, change the value of `test_trial_id` to an actual trial id.

In [ ]:
test_trial_id = "trusting_clarke_0_0"

# Trial info about a specific trial ID
info = trial_data[test_trial_id]["trial_info"]

# List of samples for specific Trial ID
samples = trial_data[test_trial_id]["samples"]
print(f"Number of samples in trial: {len(samples)}")

## Inspect Trial Data

Each trial comes with a set of generic information about the trial. It is an instance of `TrialInfo` class. 

For more information about the `TrialInfo` class, see https://cogment.ai/docs/reference/python#class-trialinfo. 

In [ ]:
print(f"Trial ID: {info.trial_id}")
print(f"Trial State: {info.trial_state}")
print(f"Number of steps: {info.sample_count}")

print(f"Environment Implementation: {print(info.parameters.environment_implementation)}")
print(f"Environment Config: {info.parameters.environment_config}")

## Inspect Sample

Each sample from the trial datastore is an instance of the `DatastoreSample` class. 

For more info on the `DatastoreSample` class, see: https://cogment.ai/docs/reference/python#class-datastoresample




In [ ]:
sample = samples[0]
print(sample)

environment_specs = EnvironmentSpecs.deserialize(info.parameters.actors[0].config.environment_specs)
action_space = environment_specs.get_action_space()
observation_space = environment_specs.get_observation_space()

# General Sample info
print(f"Trial ID: {sample.trial_id}")
print(f"Trial State: {sample.trial_state}")
print(f"Step: {sample.tick_id}")
print(f"Timestamp: {sample.timestamp}")

# Actor observation, action, reward
print(f"Actors: {list(sample.actors_data.keys())}")
cogment_obs = sample.actors_data["web_actor"].observation
cogment_action = sample.actors_data["web_actor"].action
reward = sample.actors_data["web_actor"].reward

state = observation_space.deserialize(cogment_obs).value
print(f"State: {state}")

action = action_space.deserialize(cogment_action).value
print(f"Action: {action}")

print(f"Reward: {reward}")